**Colab Link URL**
- https://colab.research.google.com/drive/14WknlsoTq9bRlJKv1v2o7A3209EhkQcy?usp=sharing

이 노트북은 [Google Colab](https://colab.research.google.com/)에서 실행하시기에 최적화되어있습니다.
상기 URL을 이용해서 코드를 실행 해보세요.

In [2]:
!pip install -q langchain-core langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

In [19]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [20]:
llm = ChatOpenAI(model="gpt-5-mini", api_key=OPENAI_API_KEY)

In [21]:
# 법률 분석을 위한 도메인 특화 프롬프트 템플릿
legal_analysis_prompt = ChatPromptTemplate.from_template(
    """
당신은 경력 20년의 IT 전문 변호사입니다. 아래 첨부된 비밀유지계약서(NDA) 초안을 '정보 제공자(Disclosing Party)'의 입장에서 검토하고, 잠재적인 법적 리스크가 있는 조항을 식별하여 그 위험성과 수정 제안을 상세히 분석해주세요.

반드시 다음의 분석 프레임워크에 따라 각 항목별로 검토 의견을 제시해야 합니다:
1.  **비밀정보의 정의 (Definition of Confidential Information):** 정의가 너무 광범위하거나 모호하여, 보호받아야 할 정보가 누락될 위험은 없는가?
2.  **비밀유지 의무 기간 (Term of Confidentiality):** 의무 기간이 '계약 종료 후 2년'으로 명시되어 있는데, 이는 당사의 핵심 영업 비밀을 보호하기에 충분한 기간인가? 기간을 '영구적'으로 수정할 필요는 없는가?
3.  **정보 반환 및 파기 (Return or Destruction of Information):** 계약 종료 시 정보 수령자의 정보 파기 의무만 명시되어 있고, 파기 사실을 증명할 의무가 누락되어 있다. 이는 위험하지 않은가?

---
[계약서 초안 텍스트]
{nda_text}
---
[법률 검토 의견]
"""
)

In [22]:
# 분석 체인 구성
analysis_chain = legal_analysis_prompt | llm | StrOutputParser()

In [23]:
# 분석 대상 NDA 텍스트
sample_nda = """
제2조 (비밀정보의 정의) "비밀정보"라 함은 본 계약과 관련하여 일방이 상대방에게 서면, 구두, 전자적 형태로 제공하는 모든 기술 및 경영상의 정보를 의미한다.
제5조 (비밀유지 의무 기간) 정보 수령자의 비밀유지 의무는 본 계약 종료 후 2년간 유효하다.
제6조 (정보의 반환) 계약이 종료되는 경우, 정보 수령자는 제공자로부터 받은 모든 비밀정보 및 그 복제물을 즉시 파기해야 한다.
"""

In [24]:
# 에이전트 실행
legal_review = analysis_chain.invoke({"nda_text": sample_nda})
print(legal_review)

아래는 정보 제공자(Disclosing Party) 관점에서 귀하가 제시한 NDA 초안의 핵심 조항 3개(비밀정보 정의 / 비밀유지 의무 기간 / 정보 반환·파기)에 대한 상세한 법률검토입니다. 각 항목별로 문제점(법적 리스크), 그 위험성, 구체적인 수정안(권장 조문 예시) 및 실무상 협상 포인트를 제시합니다.

요약(핵심 권고)
- 비밀정보 정의가 현행 단문으로는 불명확하고 예외·구체화가 부족하므로 '구체적 정의 + 제외조항 + 구두정보의 서면확인'을 추가할 것.
- 비밀유지 기간 2년은 일반 영업정보에는 단기적이며 영업비밀(Trade Secret) 보호에는 불충분. 영업비밀은 공개될 때까지 무기한 보호하도록 하고, 기타 정보는 최소 3~5년(권장 5년)으로 연장할 것.
- 반환·파기 조항에 '파기증명서 제출', '전자적 삭제방법·백업 처리', '법적보존 예외 및 통지'가 빠져 있어 추가해야 함.

1. 비밀정보의 정의 (Definition of Confidential Information)
문제점 및 위험성
- 현재 조문: “본 계약과 관련하여 일방이 상대방에게 서면, 구두, 전자적 형태로 제공하는 모든 기술 및 경영상의 정보”
- 지나치게 단문·포괄적이어서 (a) 어떤 정보가 비밀인지 명확하지 않음, (b) 구두정보 처리절차 부재, (c) 비밀정보의 범위(파생물, 분석·요약 등)가 명시되지 않아 누락 가능성, (d) 비밀정보에서 제외되어야 할 예외(공개정보, 독립개발, 제3자 취득, 사전보유 등)가 없어 분쟁시 불리할 수 있음.
- 결과적으로 정보 수령자가 "이 정보는 원래 알고 있었다 / 독립개발했다"는 주장을 쉽게 할 위험이 있고, 구두 제공 정보가 문서화되지 않으면 입증곤란.

권고(법리적 근거 포함)
- 정보의 범위(예: 소스코드, 설계도, 고객명단, 가격정책, 기술문서 등)와 파생물(분석·요약·노트 등)을 명시.
- 비밀정보에서 제외되는 항목들을 명확히 열거(공개정보, 수령자가 제공 이전에 보유·독립개발·정당한 제3자로부터 취득 등)하여 수령자